In [7]:
import json
import os
from collections import Counter
import requests
from tqdm import tqdm
from wikidata.client import Client

In [2]:
parent_folder = 'insert here your local path to the folder storing all datasets'
folder = parent_folder + 'artworks/' # choose here the name of a dataset folder

In [ ]:
def count_entities(input_folder):    
    i = 0
    for filename in tqdm(os.listdir(input_folder)):
            lista = []
            f = os.path.join(input_folder, filename)
            json_file = open(f)
            data = json.load(json_file)
            for key in data['entities']:
                i +=1
    print(i)

In [3]:
def data_subgroups(input_folder, out_folder_name, file_name, selected_classes_list, i):

    out_path = os.path.join(parent_folder, out_folder_name)

    if not os.path.exists(out_path):
        os.makedirs(out_path)

    new_data = {}

    for filename in tqdm(os.listdir(input_folder)):
        lista = []
        f = os.path.join(input_folder, filename)
        json_file = open(f)
        data = json.load(json_file)
        for key in data['entities']:
            list_of_classes = []
            for claim in data['entities'][key]['claims']['P31']:
                class_id = claim['mainsnak']['datavalue']['value']['id']
                list_of_classes.append(class_id)

            if (set(list_of_classes) & set(selected_classes_list)):
                lista.append(key)
        
        for key in lista:
            new_data.update({key:data['entities'][key]})
            while len(new_data) >= 50:
                data_to_write = {"entities": dict(list(new_data.items())[0:50]), "success": 1}
                i += 1
                with open(out_path + '/' + file_name + str(i) + '.json', 'w') as new_f:
                    json.dump(data_to_write, new_f)
                new_data = dict(list(new_data.items())[50:])
                data_to_write = {}

        for el in lista:
            del data['entities'][el]
        with open(f, "w") as jsonFile:
            json.dump(data, jsonFile)

    if len(new_data) > 0:
        i += 1
        data_to_write = {"entities": dict(list(new_data.items())[0:50]), "success": 1}
        with open(out_path + '/' + file_name + str(i) + '.json', 'w') as new_f:
                json.dump(data_to_write, new_f)   

In [ ]:
pop_cult_class_file = open(parent_folder + 'intermediate_files/pop_culture_classes.json')
pop_culture_classes = json.load(pop_cult_class_file)

data_subgroups(folder, 'pop', 'pop_culture', pop_culture_classes, 0)

In [ ]:
visual_class_file = open(parent_folder + 'intermediate_files/visual_heritage_classes.json')
visual_classes = json.load(visual_class_file)

data_subgroups(folder, 'visual', 'visual', visual_classes, 0)

In [ ]:
textual_class_file = open(parent_folder + 'intermediate_files/textual_heritage_classes.json')
textual_classes = json.load(textual_class_file)

data_subgroups(folder, 'text', 'textual', textual_classes, 0)

In [ ]:
def select_duplicates(folder):
    all_entities = []
    for filename in tqdm(os.listdir(folder)):
        f = os.path.join(folder, filename)
        json_file = open(f)
        data = json.load(json_file)
        for key in data['entities']:
            all_entities.append(key)

    counted_keys = Counter(all_entities)

    duplicates = {}
    for x,y in counted_keys.items():
        if y > 1:
            duplicates.update({x:y})

    n=0
    for k,v in duplicates.items():
        n+=v-1
    print(n) # if the printed result is 0 no duplicates exist in the dataset
        
    return counted_keys

In [ ]:
def remove_duplicates(folder, duplicates):   
    for filename in tqdm(os.listdir(folder)):
        lista = []
        f = os.path.join(folder, filename)
        json_file = open(f)
        data = json.load(json_file)
        for key in data['entities']:
            if duplicates[key] > 1:
                lista.append(key)
                duplicates = { **duplicates, key: duplicates[key]-1}
        for el in lista:
            del data['entities'][el]
            with open(f, "w") as jsonFile:
                json.dump(data, jsonFile)  

In [13]:
def compact_jsons(folder, output_folder, filename_prefix, i):
    new_data = {}
    for filename in tqdm(os.listdir(folder)):
        f = os.path.join(folder, filename)
        json_file = open(f)
        data = json.load(json_file)
        if len(data['entities']) > 0:
            new_data.update(data['entities'])
        while len(new_data) >= 50:
            data_to_write = {"entities": dict(list(new_data.items())[0:50]), "success": 1}
            i += 1
            with open(output_folder + filename_prefix + str(i) + '.json', 'w') as new_f:
                json.dump(data_to_write, new_f)
            new_data = dict(list(new_data.items())[50:])
            data_to_write = {}
                
    if len(new_data) > 0:
        i += 1
        data_to_write = {"entities": dict(list(new_data.items())[0:50]), "success": 1}
        with open(output_folder + filename_prefix + str(i) + '.json', 'w') as new_f:
                json.dump(data_to_write, new_f)                 

In [ ]:
compact_jsons(parent_folder + '/visual', 'data/visual2/', 'visual')
compact_jsons(parent_folder + '/text', 'data/textual2/', 'textual')
compact_jsons(parent_folder + '/pop', 'data/pop2/', 'pop_culture')

In [ ]:
count_entities(parent_folder + '/textual2')
count_entities(parent_folder + '/visual2')
count_entities(parent_folder + '/pop2')

In [40]:
def check_json(folder_path):

    for filename in tqdm(os.listdir(folder_path)):
        f = os.path.join(folder_path, filename)
        json_file = open(f)
        try:
            data = json.load(json_file)
        except:
            print('error occurred at:', f)
        
check_json(parent_folder + '/artworks')

  2%|█▎                                                                            | 230/13705 [00:04<04:37, 48.64it/s]

error occurred at: C:/Users/valep/OneDrive/Documenti/PhD/Conjectures/ADHO23/experiments/data//artworks\artwork10395.json


 61%|██████████████████████████████████████████████▋                              | 8319/13705 [02:48<01:32, 58.35it/s]

error occurred at: C:/Users/valep/OneDrive/Documenti/PhD/Conjectures/ADHO23/experiments/data//artworks\artwork37113.json


100%|████████████████████████████████████████████████████████████████████████████| 13705/13705 [04:49<00:00, 47.40it/s]


In [ ]:
check_json(parent_folder + '/visual2')
check_json(parent_folder + '/pop2')
check_json(parent_folder + '/text')

In [ ]:
print('#### ENTITIES IN TEXT FOLDER ######')
count_entities(parent_folder + '/text')
txt_duplicates = select_duplicates(parent_folder + '/text')
count_entities(parent_folder + '/text')

print('#### ENTITIES IN VISUAL FOLDER ######')
count_entities(parent_folder + '/visual')
viz_duplicates = select_duplicates(parent_folder + '/visual')
count_entities(parent_folder + '/visual')

print('#### ENTITIES IN POP FOLDER ######')
count_entities(parent_folder + '/pop')
pop_duplicates = select_duplicates(parent_folder + '/pop')
remove_duplicates(parent_folder + '/pop', pop_duplicates)   
count_entities(parent_folder + '/pop')

left_duplicates = select_duplicates(parent_folder + '/artworks')
remove_duplicates(parent_folder + '/artworks', left_duplicates)

In [ ]:
remove_duplicates(parent_folder + '/text', txt_duplicates)   
remove_duplicates(parent_folder + '/visual', viz_duplicates)  
remove_duplicates(parent_folder + '/pop', pop_duplicates)   
remove_duplicates(parent_folder + '/artworks', left_duplicates)

In [ ]:
count_entities(parent_folder + '/text')
count_entities(parent_folder + '/visual')
count_entities(parent_folder + '/pop')
count_entities(parent_folder + '/artworks')

In [23]:
temp_dict = {}
for filename in tqdm(os.listdir(parent_folder + '/artworks')):
    f = os.path.join(parent_folder + '/artworks', filename)
    json_file = open(f)
    data = json.load(json_file)
    for key in data['entities']:
        for claim in data['entities'][key]['claims']['P31']:
            class_id = claim['mainsnak']['datavalue']['value']['id']
            if class_id not in temp_dict:
                temp_dict.update({class_id:1})
            else:
                temp_dict.update({class_id:temp_dict[class_id] + 1})
                
sorted_classes = sorted(temp_dict.items(), key=lambda x:x[1], reverse=True)    

# xd = open("query.json")
# xdd = json.load(xd)
# for x in sorted_classes:
#     for y in xdd:
#         value = y['item'].replace('http://www.wikidata.org/entity/', '')
#         if x[0] == value:
#             print("\"" + value + "\",")
    

# for x in sorted_classes:
#     for y in theatrica_genre:
#         if x[0] == y:
#             print("\"" + y + "\",")

print(len(sorted_classes))
client = Client() 
for x in sorted_classes:
    if x[1] < 15 and x[1] > 0:
        try:
            entity = client.get(x[0], load=True)
            print(x, entity.label.texts['en'])
        except:
            print(x)

100%|██████████████████████████████████████████████████████████████████████████████| 1737/1737 [00:27<00:00, 63.46it/s]


In [27]:
def remove_empty_files(folder):   
    
    lista = []
    for filename in tqdm(os.listdir(folder)):
        f = os.path.join(folder, filename)
        json_file = open(f)
        print(filename)
        data = json.load(json_file)
        empty_dict = {'entities': {}, 'success': 1}
        if data == empty_dict:
            lista.append(filename)
        
    for fn in tqdm(lista):
        os.remove(os.path.join(folder, fn))

In [ ]:
irrelevant_classes = [x[0] for x in sorted_classes if x[1] > 800]

data_subgroups(folder, 'others', 'other', irrelevant_classes, 0)

In [ ]:
remove_empty_files(parent_folder + '/artworks')

In [12]:
pop_cult_class_file = open(parent_folder + 'intermediate_files/pop_culture_classes.json')
pop_culture_classes = json.load(pop_cult_class_file)

data_subgroups(folder, 'new_pop', 'pop_culture', pop_culture_classes, 25035)

visual_class_file = open(parent_folder + 'intermediate_files/visual_heritage_classes.json')
visual_classes = json.load(visual_class_file)

data_subgroups(folder, 'new_viz', 'visual', visual_classes, 21553)

textual_class_file = open(parent_folder + 'intermediate_files/textual_heritage_classes.json')
textual_classes = json.load(textual_class_file)

data_subgroups(folder, 'new_tex', 'textual', textual_classes, 12505)

100%|██████████████████████████████████████████████████████████████████████████████| 1737/1737 [02:05<00:00, 13.86it/s]


In [17]:
#compact_jsons(parent_folder + '/new_viz', 'data/new_viz2/', 'visual', 21518)
#compact_jsons(parent_folder + '/new_tex', 'data/new_tex2/', 'textual', 12468)
compact_jsons(parent_folder + '/new_pop', 'data/new_pop2/', 'pop_culture', 25018)

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:01<00:00,  8.57it/s]


In [22]:
def copy_jsons(folder, output_folder, filename_prefix):
    i = 1
    for filename in tqdm(os.listdir(folder)):
        f = os.path.join(folder, filename)
        json_file = open(f)
        data = json.load(json_file)
        with open(output_folder + filename_prefix + str(i) + '.json', 'w') as new_f:
                json.dump(data, new_f)
        i += 1

copy_jsons(parent_folder + '/pop', 'data/new_pop/', 'pop_culture')
copy_jsons(parent_folder + '/visual', 'data/new_viz/', 'visual')
copy_jsons(parent_folder + '/textual', 'data/new_text/', 'textual')